## Imports

In [ ]:
# Built-in imports
import os
if sys.version_info[0] < 3:
    from Tkinter import Tk
    import TkFileDialog as filedialog
else:
    from Tkinter import Tk
    from Tkinter import filedialog

# third-party imports
import numpy as np  # noqa: E402
import matplotlib.pyplot as plt  # noqa: E402
import cv2  # noqa: E402

# Custom module imports
from split_image_into_blocks import split_image_into_blocks  # noqa: E402
from process_block_image import process_block_image  # noqa: E402

## Get scanned image and split it into blocks

In [ ]:
root = Tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename(
    initialdir='../../Images/Scanner Images/')
base_file = os.path.splitext(file_path)[0]

img_list = []
block_size = (1024, 1024)
status, base_mean = split_image_into_blocks(file_path, img_list, block_size)
if len(status) != 0:
    print(status)

## Process all blocks and display

In [ ]:
counter = 0
block_count = 0
show_plots = True # Toggle display
obj = {'sizes': [], 'detected': []}
count_list = []
for block_image in img_list:
    b_img = block_image.copy()
    processed_image, count = process_block_image(b_img, color, count_list,
                                                 obj=obj, retr_objects=False)
    counter += count
    block_count += 1
    print("Image Block %s Egg count %s" % (block_count, count))

    if show_plots:
        plt.subplot(1, 2, 1)
        plt.imshow(block_image)
        plt.title('Original Image'), plt.xticks([]), plt.yticks([])

        plt.subplot(1, 2, 2)
        plt.imshow(processed_image)
        plt.title('Processed Image'), plt.xticks([]), plt.yticks([])

        plt.show()
        plt.waitforbuttonpress(timeout=-1)
        
print(counter)

## Detect eggs in a single block

In [ ]:
block_no = 113
b_img = img_list[block_no].copy()
color = (0, 255, 255)
count = 0
hsv_low = np.array([140, 135, 155], dtype=np.float32) / 255
hsv_high = np.array([165, 255, 225], dtype=np.float32) / 255
# base_mean = (0.53, 0.07, 0.88)
# b_img = cv2.medianBlur(b_img,5)
b_img_hsv = np.float32(cv2.cvtColor(b_img, cv2.COLOR_BGR2HSV))
b_img_hsv = b_img_hsv / np.max(b_img_hsv)
BW = np.uint8(cv2.split(b_img_hsv)[1] > 0.5)
#    b_image = cv2.bitwise_and(b_img,b_img,mask=BW)
b_image_hsv = cv2.bitwise_and(b_img_hsv, b_img_hsv, mask=BW)

mean = np.round(np.array(cv2.mean(b_img_hsv))[:-1], 3)
tolerance_factor = (0.05, 1.5, 7)
tolerance = np.array((1 + tolerance_factor * (mean - base_mean) / base_mean),
                     dtype=np.float32)
# apply the hsv range on a mask
mask = cv2.inRange(b_image_hsv, hsv_low, hsv_high)
_, contours, _ = cv2.findContours(mask, cv2.RETR_TREE,
                                  cv2.CHAIN_APPROX_NONE)
area = [cv2.contourArea(cnt) for cnt in contours]
for k in np.arange(len(contours)):
    cnt = contours[k]
    if 40 <= cv2.contourArea(cnt) <= 135:
        #        rect = cv2.minAreaRect(cnt)
        rect = cv2.minAreaRect(cnt)
        (object_w, object_h) = (round(max(rect[1]), 2),
                                round(min(rect[1]), 2))
        print(k, object_w, object_h, round(object_w / object_h, 2))
        if 1.78 <= round(object_w / object_h, 2) <= 3.35 \
                and 12 < object_w < 22:
            print(object_w, object_h, round(object_w / object_h, 2))
            count += 1
    else:
        continue

## Show original and processed block side by side

In [ ]:
block_no = 113
count_list = []
show_plots = True
b_img = img_list[block_no].copy()
obj = {'sizes': [], 'detected': []}
processed_image, count = process_block_image(b_img, color, count_list,
                                             obj=obj, retr_objects=False)
print("Egg count %s" % count)
# plt.ioff()

if show_plots:
    fig = plt.figure(figsize=(8.0, 5.0))
    fig.add_subplot(121).imshow(b_img)
    fig.add_subplot(121).set_title('Original Image'), plt.xticks(
        []), plt.yticks([])
    fig.add_subplot(122).imshow(processed_image)
    fig.add_subplot(122).set_title('Egg count %s' % count), plt.xticks(
        []), plt.yticks([])

    fig.show()
    try:
        fig.waitforbuttonpress(timeout=-1)
        base_file = os.path.splitext(file_path)[0]
    # fig.savefig(base_file+'1'+'.png', bbox_inches='tight', dpi=200)
    except Tk.TclError:
        print("Program exited")